In [ ]:
# instalar o plotly
!pip install plotly==4.8.1

In [ ]:
import pandas as pd
import numpy as np
# Esta linha de comando altera o backend de renderização de gráficos do matplotlib para o Ploty (que possui graficos mais detalhados e bonitos)
pd.options.plotting.backend = "plotly"

# Carregar Dataset Movielens

https://grouplens.org/datasets/movielens/


Este Exemplo: https://grouplens.org/datasets/movielens/10m/


In [ ]:
# Carregar um arquivo csv para um dataframe
# Documentação do comando: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html
# Carregando CSV dos filmes
df_filmes = pd.read_csv('/content/drive/My Drive/Colab Notebooks (1)/mineracao/sistema_recomendacao/filmes/movies.dat', sep='::', names=['id_filme', 'nome', 'categoria'])
df_filmes.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: ParserWarning:

Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.



,id_filme,nome,categoria
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
df_ava = pd.read_csv( '/content/drive/My Drive/Colab Notebooks (1)/mineracao/sistema_recomendacao/filmes/ratings.dat', sep= '::', names=['id_usuario','id_filme','avaliacao','timestamp'] )
df_ava.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: ParserWarning:

Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.



,id_usuario,id_filme,avaliacao,timestamp
0,1,122,5.0,838985046
1,1,185,5.0,838983525
2,1,231,5.0,838983392
3,1,292,5.0,838983421
4,1,316,5.0,838983392


In [ ]:
# Carregando csv das tags
df_tags = pd.read_csv( '/content/drive/My Drive/Colab Notebooks (1)/mineracao/sistema_recomendacao/filmes/tags.dat', sep= '::', names=['id_usuario','id_filme','tag','timestamp'] )
df_tags.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: ParserWarning:

Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.



,id_usuario,id_filme,tag,timestamp
0,15,4973,excellent!,1215184630
1,20,1747,politics,1188263867
2,20,1747,satire,1188263867
3,20,2424,chick flick 212,1188263835
4,20,2424,hanks,1188263835


# Ajustando as datas

Unix Timestamp -> Datetime

In [ ]:
# Transformando Unix timestamp no formato padrão datetime do pandas
# coumentação: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.to_datetime.html
df_tags['data'] = pd.to_datetime(df_tags['timestamp'], unit='s').dt.date
df_tags.head()

,id_usuario,id_filme,tag,timestamp,data
0,15,4973,excellent!,1215184630,2008-07-04
1,20,1747,politics,1188263867,2007-08-28
2,20,1747,satire,1188263867,2007-08-28
3,20,2424,chick flick 212,1188263835,2007-08-28
4,20,2424,hanks,1188263835,2007-08-28


In [ ]:
df_ava['data'] = pd.to_datetime(df_ava['timestamp'], unit='s').dt.date
df_ava.head()

,id_usuario,id_filme,avaliacao,timestamp,data
0,1,122,5.0,838985046,1996-08-02
1,1,185,5.0,838983525,1996-08-02
2,1,231,5.0,838983392,1996-08-02
3,1,292,5.0,838983421,1996-08-02
4,1,316,5.0,838983392,1996-08-02


# Sumarização dos Datasets

In [ ]:
# Verificando a dimensão do dataframe (linhas, colunas)
print( 'Tabela de avaliações do usúarios' )
print( df_ava.shape )

Tabela de avaliações do usúarios
(10000054, 5)


In [ ]:
print( 'Tabela de filmes e categorias' )
print( df_filmes.shape )

Tabela de filmes e categorias
(10681, 3)


In [ ]:
print( 'Tabela de Tags' )
print( df_tags.shape )

Tabela de Tags
(95580, 5)


# Análise descritiva


In [ ]:
n_usuarios = df_ava['id_usuario'].nunique() # Valeu Tegão R.
n_filmes = df_filmes['id_filme'].nunique()
print('usuários: {}'.format(n_usuarios))
print('filmes: {}'.format(n_filmes))

usuários: 69878
filmes: 10681


## Questões à serem respondidas

In [ ]:
# Quais usuários mais/menos avaliaram filmes?  (Mostre os top 5)

# Quais usuários mais/menos exigentes? (Top 10 haters/happy users)

# Quais os filmes mais/menos assistidos (avaliados)?

# Quais os filmes não tiveram nenhuma avaliação?

# Quais os melhores/piores filmes? (Com base na opnião dos usuários)

# Quais gêneros foram produzidos mais/menos filmes?

# Quais os gêneros mais/menos populares entre os usuários?

# Quais os melhores/piores filmes de cada gênero? (Com base na opnião dos usuários) (alunos)

## Exploração Inicial e Filtros (dataset de avaliações)

## Quais usuários mais/menos avaliaram filmes?  (Mostre os top 5)

In [ ]:
# Verificando quantas avaliações foram feitas por cada usuário:
# .value_counts() -> dica do Henrique Saboya (valeu!), mais eficiente que realizar um groupby de count em uma unica coluna
df_qtd_ava = df_ava.id_usuario.value_counts()
df_qtd_ava

59269    7359
67385    7047
14463    5169
68259    4483
27468    4449
         ... 
41151      20
15833      20
10531      20
43559      20
12546      20
Name: id_usuario, Length: 69878, dtype: int64

In [ ]:
# Quem mais votou?
df_qtd_ava.nlargest()

59269    7359
67385    7047
14463    5169
68259    4483
27468    4449
Name: id_usuario, dtype: int64

In [ ]:
# Quem menos votou?
df_qtd_ava.nsmallest()

2773     20
57408    20
14391    20
16780    20
653      20
Name: id_usuario, dtype: int64

In [ ]:
# Média
df_qtd_ava.mean()

143.10732991785684

In [ ]:
# Mediana
df_qtd_ava.median()

69.0

## Boxplot

In [ ]:
fig = df_qtd_ava.plot( kind='box', title='Qtd. de Avaliações por usúario' )
fig.show()

In [ ]:
# Todas as métricas
df_qtd_ava.describe()

count    69878.00000
mean       143.10733
std        216.71258
min         20.00000
25%         35.00000
50%         69.00000
75%        156.00000
max       7359.00000
Name: id_usuario, dtype: float64

In [ ]:
import plotly.express as px

fig = px.histogram(df_qtd_ava, marginal="box")
fig.show()

In [ ]:
# Se eu for cortar DF em algum valor, quanto eu perco?
n_superior = 0.97
print('Valor para corte superior', df_qtd_ava.quantile( n_superior ) ) # 97% <

Valor para corte superior 665.6900000000023


In [ ]:
# Se eu for cortar DF em algum valor, quanto eu perco?
print('Valor para corte inferior', df_qtd_ava.quantile( 0.01) ) # 1% >

Valor para corte inferior 20.0


In [ ]:
# Aplicando filtro
n_filtro = df_qtd_ava.quantile( n_superior )
df_qtd_ava_filtrado = df_qtd_ava[ df_qtd_ava <= n_filtro ]
df_qtd_ava_filtrado

12150    665
14928    665
47990    665
71231    665
46024    665
        ... 
41151     20
15833     20
10531     20
43559     20
12546     20
Name: id_usuario, Length: 67781, dtype: int64

In [ ]:
import plotly.express as px

fig = px.histogram(df_qtd_ava_filtrado, marginal="box")
fig.show()

In [ ]:
df_qtd_ava_filtrado.describe()

count    67781.000000
mean       114.934952
std        121.525629
min         20.000000
25%         35.000000
50%         66.000000
75%        143.000000
max        665.000000
Name: id_usuario, dtype: float64

Filtrar os outliers do DF original de avaliações

In [ ]:
lista_usu = list( df_qtd_ava_filtrado.index )
print(lista_usu)

[12150, 14928, 47990, 71231, 46024, 24057, 21544, 71149, 2034, 26484, 28568, 29725, 65614, 51459, 2675, 60556, 56386, 11028, 8127, 58006, 12006, 70769, 4333, 53480, 17193, 32612, 2517, 11858, 33903, 48144, 25863, 5798, 40148, 17276, 18090, 46622, 18086, 63458, 31380, 26152, 57902, 31487, 32444, 44177, 21414, 11032, 70478, 8060, 69138, 33957, 57750, 33521, 42388, 16164, 18833, 44585, 10821, 53545, 9877, 6245, 61841, 15685, 52340, 65738, 44727, 48210, 10112, 43762, 70034, 64821, 48107, 35303, 71036, 9844, 5467, 15609, 64311, 62541, 38110, 25008, 39604, 6814, 35396, 61915, 1536, 47553, 46890, 2810, 49821, 46984, 62166, 20410, 2429, 70650, 4317, 62275, 23174, 31251, 60716, 14356, 12760, 52334, 37271, 29937, 37419, 41594, 40119, 2714, 67853, 40334, 20306, 46505, 10036, 867, 13740, 50481, 8041, 58264, 42321, 5118, 60081, 48681, 13453, 52359, 66746, 58470, 31349, 23493, 31429, 62236, 13815, 18161, 50149, 1751, 28986, 10270, 37369, 54113, 51813, 54937, 44264, 27505, 26247, 26465, 64317, 42874,

In [ ]:
df_ava_fil = df_ava[ df_ava.id_usuario.isin( lista_usu ) ]
df_ava_fil.head()

,id_usuario,id_filme,avaliacao,timestamp,data
0,1,122,5.0,838985046,1996-08-02
1,1,185,5.0,838983525,1996-08-02
2,1,231,5.0,838983392,1996-08-02
3,1,292,5.0,838983421,1996-08-02
4,1,316,5.0,838983392,1996-08-02


In [ ]:
print( df_ava.shape )
print( df_ava_fil.shape )

(10000054, 5)
(7790406, 5)


In [ ]:
df_ava_fil.reset_index(drop=True, inplace=True)
df_ava_fil

,id_usuario,id_filme,avaliacao,timestamp,data
0,1,122,5.0,838985046,1996-08-02
1,1,185,5.0,838983525,1996-08-02
2,1,231,5.0,838983392,1996-08-02
3,1,292,5.0,838983421,1996-08-02
4,1,316,5.0,838983392,1996-08-02
...,...,...,...,...,...
7790401,71567,2107,1.0,912580553,1998-12-02
7790402,71567,2126,2.0,912649143,1998-12-03
7790403,71567,2294,5.0,912577968,1998-12-02
7790404,71567,2338,2.0,912578016,1998-12-02


## Quais usuários mais/menos exigentes? (Top 10 haters/happy users)

In [ ]:
df_ava_fil.head(3)

,id_usuario,id_filme,avaliacao,timestamp,data
0,1,122,5.0,838985046,1996-08-02
1,1,185,5.0,838983525,1996-08-02
2,1,231,5.0,838983392,1996-08-02


In [ ]:
# Calcular a média de avaliação de cada usuário
df_ava_fil.groupby( by = 'id_usuario' ).mean()

,id_filme,avaliacao,timestamp
id_usuario,,,
1,411.681818,5.000000,8.389840e+08
2,785.550000,3.200000,8.682453e+08
3,5441.757576,3.939394,1.140207e+09
4,355.289474,4.026316,8.444169e+08
5,653.827586,3.850575,8.579125e+08
...,...,...,...
71563,8214.069767,3.441860,1.162134e+09
71564,2792.800000,2.980952,1.111545e+09
71565,1928.731034,3.889655,9.743113e+08


In [ ]:
# Filtrar depois
df_haters = df_ava_fil.groupby( by = 'id_usuario' ).mean()['avaliacao']
df_haters

id_usuario
1        5.000000
2        3.200000
3        3.939394
4        4.026316
5        3.850575
           ...   
71563    3.441860
71564    2.980952
71565    3.889655
71566    3.836735
71567    3.207547
Name: avaliacao, Length: 67781, dtype: float64

In [ ]:
# Qtd de filmes avaliados
df_ava_fil_qtd = df_ava_fil[['id_usuario', 'avaliacao']].groupby( by='id_usuario' ).count()
df_ava_fil_qtd

,avaliacao
id_usuario,
1,22
2,20
3,33
4,38
5,87
...,...
71563,43
71564,105
71565,145


In [ ]:
df_ava_fil_qtd.rename( columns={'avaliacao':'qtd'}, inplace=True )
df_ava_fil_qtd

,qtd
id_usuario,
1,22
2,20
3,33
4,38
5,87
...,...
71563,43
71564,105
71565,145


In [ ]:
df_haters_final = df_ava_fil_qtd.join( df_haters )
df_haters_final.head(3)

,qtd,avaliacao
id_usuario,,
1,22,5.000000
2,20,3.200000
3,33,3.939394


In [ ]:
print(df_haters_final.sort_values( by=['avaliacao', 'qtd'], ascending=False ).head(10))
print(df_haters_final.sort_values( by=['avaliacao', 'qtd'], ascending=False ).tail(10))

            qtd  avaliacao
id_usuario                
52749        97        5.0
68379        65        5.0
18965        53        5.0
13027        35        5.0
15575        31        5.0
54009        30        5.0
35184        24        5.0
1            22        5.0
22045        22        5.0
11884        21        5.0
            qtd  avaliacao
id_usuario                
24176       147   1.000000
3457         20   1.000000
24490        20   1.000000
8920         20   0.950000
6322         21   0.785714
63381        22   0.522727
48146        31   0.500000
13496        20   0.500000
49862        20   0.500000
62815        20   0.500000


## Quais os filmes mais/menos assistidos (avaliados)?

In [ ]:
# relembrando o dataser
df_ava_fil.head(3)

,id_usuario,id_filme,avaliacao,timestamp,data
0,1,122,5.0,838985046,1996-08-02
1,1,185,5.0,838983525,1996-08-02
2,1,231,5.0,838983392,1996-08-02


In [ ]:
df_mais_avaliados = df_ava_fil.id_filme.value_counts().rename('qtd_avaliada')
df_mais_avaliados.head()

296    32900
356    32484
593    31690
480    30658
318    29251
Name: qtd_avaliada, dtype: int64

In [ ]:
df_mais_avaliados.tail()

57848    1
8679     1
53752    1
33264    1
3151     1
Name: qtd_avaliada, dtype: int64

In [ ]:
df_filmes.head(3)

,id_filme,nome,categoria
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance


In [ ]:
# Verificando manualmente o filme 296
df_filmes.query('id_filme == 296')

,id_filme,nome,categoria
293,296,Pulp Fiction (1994),Comedy|Crime|Drama


In [ ]:
df_mais_avaliados

296      32900
356      32484
593      31690
480      30658
318      29251
         ...  
57848        1
8679         1
53752        1
33264        1
3151         1
Name: qtd_avaliada, Length: 10328, dtype: int64

In [ ]:
df_mais_avaliados_filmes = df_mais_avaliados.to_frame().merge( df_filmes[['id_filme', 'nome']], how='left', left_index=True, right_on='id_filme' )
df_mais_avaliados_filmes

,qtd_avaliada,id_filme,nome
293,32900,296,Pulp Fiction (1994)
352,32484,356,Forrest Gump (1994)
587,31690,593,"Silence of the Lambs, The (1991)"
476,30658,480,Jurassic Park (1993)
315,29251,318,"Shawshank Redemption, The (1994)"
...,...,...,...
10171,1,57848,Canvas (2006)
7976,1,8679,"Hunters, The (1958)"
9925,1,53752,Gui Si (Silk) (2006)
9083,1,33264,Satan's Tango (Sátántangó) (1994)


In [ ]:
# Quantos filmes avaliaram menos de 5 vezes?
N = 5
df_mais_avaliados_filmes[ df_mais_avaliados_filmes.qtd_avaliada.lt(N+1) ]

,qtd_avaliada,id_filme,nome
4292,5,4385,Pandaemonium (2000)
1749,5,1830,Follow the Bitch (1996)
7885,5,8587,Mayor of the Sunset Strip (2003)
8944,5,32074,Kiss the Bride (2002)
3236,5,3321,"Waiting Game, The (2000)"
...,...,...,...
10171,1,57848,Canvas (2006)
7976,1,8679,"Hunters, The (1958)"
9925,1,53752,Gui Si (Silk) (2006)
9083,1,33264,Satan's Tango (Sátántangó) (1994)


## Quais os filmes não tiveram nenhuma avaliação?

In [ ]:
# Relembrando
df_ava.head(3)

,id_usuario,id_filme,avaliacao,timestamp,data
0,1,122,5.0,838985046,1996-08-02
1,1,185,5.0,838983525,1996-08-02
2,1,231,5.0,838983392,1996-08-02


In [ ]:
# Relembrando
filmes_assistidos = df_ava.id_filme.unique()
filmes_assistidos

array([  122,   185,   231, ..., 64652, 64427, 63662])

In [ ]:
df_filmes[ ~df_filmes.id_filme.isin(filmes_assistidos) ]

,id_filme,nome,categoria
8391,25942,Louisiana Story (1948),Drama
10360,60566,Just Another Love Story (Kærlighed på film) (2...,Crime|Drama|Thriller
10510,62669,Black River (Kuroi kawa) (1957),Drama
10661,64959,"Divide and Conquer (Why We Fight, 3) (1943)",Documentary|War


## Quais os melhores/piores filmes? (Com base na opnião dos usuários)

In [ ]:
# Relembrando
df_ava.head(3)

,id_usuario,id_filme,avaliacao,timestamp,data
0,1,122,5.0,838985046,1996-08-02
1,1,185,5.0,838983525,1996-08-02
2,1,231,5.0,838983392,1996-08-02


In [ ]:
# eliminar filmes com menos de N avaliações
N = 50
df_mais_avaliados_filmes[df_mais_avaliados_filmes.qtd_avaliada.gt( N ) ]

,qtd_avaliada,id_filme,nome
293,32900,296,Pulp Fiction (1994)
352,32484,356,Forrest Gump (1994)
587,31690,593,"Silence of the Lambs, The (1991)"
476,30658,480,Jurassic Park (1993)
315,29251,318,"Shawshank Redemption, The (1994)"
...,...,...,...
10003,51,55071,No End in Sight (2007)
5425,51,5521,"Principal, The (1987)"
1609,51,1666,Hugo Pool (1997)
5641,51,5739,Faces of Death 6 (1996)


In [ ]:
lista_filmes_N = df_mais_avaliados_filmes[ df_mais_avaliados_filmes.qtd_avaliada.gt( N ) ].id_filme.values
lista_filmes_N

array([ 296,  356,  593, ..., 1666, 5739, 7261])

In [ ]:
df_ava_aux = df_ava[ df_ava.id_filme.isin(lista_filmes_N) ]
df_ava_aux.head()

,id_usuario,id_filme,avaliacao,timestamp,data
0,1,122,5.0,838985046,1996-08-02
1,1,185,5.0,838983525,1996-08-02
2,1,231,5.0,838983392,1996-08-02
3,1,292,5.0,838983421,1996-08-02
4,1,316,5.0,838983392,1996-08-02


In [ ]:
# df_ava_aux
df_melhores = df_ava_aux[ ['id_filme', 'avaliacao'] ].groupby(by='id_filme').mean().sort_values(by='avaliacao', ascending=False)
df_melhores

,avaliacao
id_filme,
318,4.457238
858,4.415085
50,4.367142
527,4.363483
922,4.321966
...,...
1826,1.175214
6587,1.174716
4775,1.167105


In [ ]:
# relembrando
df_filmes.head(3)

,id_filme,nome,categoria
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance


In [ ]:
pd.merge( df_melhores, df_filmes[['id_filme', 'nome']], how='left', left_index=True, right_on='id_filme' )

,avaliacao,id_filme,nome
315,4.457238,318,"Shawshank Redemption, The (1994)"
843,4.415085,858,"Godfather, The (1972)"
49,4.367142,50,"Usual Suspects, The (1995)"
523,4.363483,527,Schindler's List (1993)
905,4.321966,922,Sunset Blvd. (a.k.a. Sunset Boulevard) (1950)
...,...,...,...
1746,1.175214,1826,Barney's Great Adventure (1998)
6475,1.174716,6587,Gigli (2003)
4681,1.167105,4775,Glitter (2001)
6264,1.009615,6371,Pokémon Heroes (2003)


## Quais gêneros foram produzidos mais/menos filmes?

In [ ]:
# relembrando
df_filmes.head(3)

,id_filme,nome,categoria
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance


In [ ]:
df_cat_temp = df_filmes.categoria.str.split('|', expand=True)
df_cat_temp

,0,1,2,3,4,5,6,7
0,Adventure,Animation,Children,Comedy,Fantasy,None,None,None
1,Adventure,Children,Fantasy,None,None,None,None,None
2,Comedy,Romance,None,None,None,None,None,None
3,Comedy,Drama,Romance,None,None,None,None,None
4,Comedy,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...
10676,Adventure,Children,Comedy,None,None,None,None,None
10677,Crime,Drama,Romance,None,None,None,None,None
10678,Comedy,Drama,None,None,None,None,None,None
10679,Drama,Romance,None,None,None,None,None,None


In [ ]:
# Lista com todos os generos que aparecem no dataset
conjunto_cat = set( df_cat_temp[0].unique() )
for i in range(1, 8):
    conjunto_cat.union( set(df_cat_temp[i].unique()) )

conjunto_cat

{'(no genres listed)',
 'Action',
 'Adventure',
 'Animation',
 'Children',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Fantasy',
 'Film-Noir',
 'Horror',
 'IMAX',
 'Musical',
 'Mystery',
 'Romance',
 'Sci-Fi',
 'Thriller',
 'War',
 'Western'}

In [ ]:
conjunto_cat.remove( '(no genres listed)' )
conjunto_cat.remove( 'IMAX' )
print(conjunto_cat)

{'Musical', 'Thriller', 'Adventure', 'War', 'Animation', 'Children', 'Comedy', 'Sci-Fi', 'Documentary', 'Mystery', 'Horror', 'Film-Noir', 'Western', 'Fantasy', 'Drama', 'Romance', 'Action', 'Crime'}


Iremos utilizar o CountVectorizer do sklearn para criar um vocabulário (dicionário) para cada categoria, e depois disso utilizar esse vocabulário para criar um vetor com 1 na categoria do filme e 0 nas outras (one-hot-encoding)

In [ ]:
voc_dict = {}

i = 0
for gen in conjunto_cat:
    voc_dict[ gen.lower() ] = i
    i = i + 1

voc_dict

{'action': 16,
 'adventure': 2,
 'animation': 4,
 'children': 5,
 'comedy': 6,
 'crime': 17,
 'documentary': 8,
 'drama': 14,
 'fantasy': 13,
 'film-noir': 11,
 'horror': 10,
 'musical': 0,
 'mystery': 9,
 'romance': 15,
 'sci-fi': 7,
 'thriller': 1,
 'war': 3,
 'western': 12}

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer( analyzer = 'word', vocabulary=voc_dict, ngram_range=(1,2), token_pattern = '[a-zA-Z0-9$&+,:;=?@#<>.^*()%!-]+' )


In [ ]:
# vocabulario
vectorizer.fit( conjunto_cat )

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 2), preprocessor=None, stop_words=None,
                strip_accents=None,
                token_pattern='[a-zA-Z0-9$&+,:;=?@#<>.^*()%!-]+',
                tokenizer=None,
                vocabulary={'action': 16, 'adventure': 2, 'animation': 4,
                            'children': 5, 'comedy': 6, 'crime': 17,
                            'documentary': 8, 'drama': 14, 'fantasy': 13,
                            'film-noir': 11, 'horror': 10, 'musical': 0,
                            'mystery': 9, 'romance': 15, 'sci-fi': 7,
                            'thriller': 1, 'war': 3, 'western': 12})

In [ ]:
vectorizer.vocabulary_

{'action': 16,
 'adventure': 2,
 'animation': 4,
 'children': 5,
 'comedy': 6,
 'crime': 17,
 'documentary': 8,
 'drama': 14,
 'fantasy': 13,
 'film-noir': 11,
 'horror': 10,
 'musical': 0,
 'mystery': 9,
 'romance': 15,
 'sci-fi': 7,
 'thriller': 1,
 'war': 3,
 'western': 12}

In [ ]:
df_filmes.categoria

0        Adventure|Animation|Children|Comedy|Fantasy
1                         Adventure|Children|Fantasy
2                                     Comedy|Romance
3                               Comedy|Drama|Romance
4                                             Comedy
                            ...                     
10676                      Adventure|Children|Comedy
10677                            Crime|Drama|Romance
10678                                   Comedy|Drama
10679                                  Drama|Romance
10680                                         Comedy
Name: categoria, Length: 10681, dtype: object

In [ ]:
# [ 0,0 ,0 ,1 ,0 ,1 ,0 ,0 ,0 ,1 ,0 ,1 ,0 ,0 ,0 ,0, 1, 0, 0 ]  <- 18 posições
# [ 0,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,1 ,0 ,0 ,0 ,0, 0, 0, 0 ]  <- 18 posições

In [ ]:
matriz_cat = vectorizer.transform( df_filmes.categoria )
matriz_cat

<10681x18 sparse matrix of type '<class 'numpy.int64'>'
	with 21534 stored elements in Compressed Sparse Row format>

In [ ]:
matriz_cat.todense()

matrix([[0, 0, 1, ..., 0, 0, 0],
        [0, 0, 1, ..., 0, 0, 0],
        [0, 0, 0, ..., 1, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 1, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
# nome das colunas
vectorizer.get_feature_names()

['musical',
 'thriller',
 'adventure',
 'war',
 'animation',
 'children',
 'comedy',
 'sci-fi',
 'documentary',
 'mystery',
 'horror',
 'film-noir',
 'western',
 'fantasy',
 'drama',
 'romance',
 'action',
 'crime']

In [ ]:
df_categoria_onehot = pd.DataFrame(data=matriz_cat.todense(), columns=vectorizer.get_feature_names() )
df_categoria_onehot

,musical,thriller,adventure,war,animation,children,comedy,sci-fi,documentary,mystery,horror,film-noir,western,fantasy,drama,romance,action,crime
0,0,0,1,0,1,1,1,0,0,0,0,0,0,1,0,0,0,0
1,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0
3,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0
4,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10676,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0
10677,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1
10678,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0
10679,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0


In [ ]:
df_filmes.head()

,id_filme,nome,categoria
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
df_filmes_cat_onehot =  pd.merge( df_filmes, df_categoria_onehot, how='left', left_index=True, right_index=True )
df_filmes_cat_onehot.head()

,id_filme,nome,categoria,musical,thriller,adventure,war,animation,children,comedy,sci-fi,documentary,mystery,horror,film-noir,western,fantasy,drama,romance,action,crime
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,0,0,1,0,1,1,1,0,0,0,0,0,0,1,0,0,0,0
1,2,Jumanji (1995),Adventure|Children|Fantasy,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0
2,3,Grumpier Old Men (1995),Comedy|Romance,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0
4,5,Father of the Bride Part II (1995),Comedy,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
df_gen_mais_menos = df_filmes_cat_onehot.iloc[:,3:].sum().sort_values(ascending=False)
df_gen_mais_menos.head()

drama       5339
comedy      3703
thriller    1706
romance     1685
action      1473
dtype: int64

In [ ]:
df_gen_mais_menos.tail()

documentary    482
musical        436
animation      286
western        275
film-noir      148
dtype: int64

In [ ]:
df_gen_mais_menos.head().plot.bar(title="Gênero com mais filmes produzidos")

In [ ]:
df_gen_mais_menos.tail().plot.bar(title="Gênero com menos filmes produzidos")

## Quais os gêneros mais/menos populares entre os usuários?

In [ ]:
# relembrando
df_ava.head(3)

,id_usuario,id_filme,avaliacao,timestamp,data
0,1,122,5.0,838985046,1996-08-02
1,1,185,5.0,838983525,1996-08-02
2,1,231,5.0,838983392,1996-08-02


In [ ]:
# relembrando
df_filmes.head(3)

,id_filme,nome,categoria
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance


In [ ]:
# relembrando
df_filmes_cat_onehot.head(3)

,id_filme,nome,categoria,musical,thriller,adventure,war,animation,children,comedy,sci-fi,documentary,mystery,horror,film-noir,western,fantasy,drama,romance,action,crime
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,0,0,1,0,1,1,1,0,0,0,0,0,0,1,0,0,0,0
1,2,Jumanji (1995),Adventure|Children|Fantasy,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0
2,3,Grumpier Old Men (1995),Comedy|Romance,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0


In [ ]:
df_filmes_cat_onehot.query( 'id_filme == 185' )

,id_filme,nome,categoria,musical,thriller,adventure,war,animation,children,comedy,sci-fi,documentary,mystery,horror,film-noir,western,fantasy,drama,romance,action,crime
183,185,"Net, The (1995)",Action|Crime|Thriller,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1


Merge na lista de avaliações com o df de filmes e categorias em one-hot

In [ ]:
df_ava[['id_usuario', 'id_filme', 'avaliacao']].head(3)

,id_usuario,id_filme,avaliacao
0,1,122,5.0
1,1,185,5.0
2,1,231,5.0


In [ ]:
df_filmes_cat_onehot.head(3)

,id_filme,nome,categoria,musical,thriller,adventure,war,animation,children,comedy,sci-fi,documentary,mystery,horror,film-noir,western,fantasy,drama,romance,action,crime
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,0,0,1,0,1,1,1,0,0,0,0,0,0,1,0,0,0,0
1,2,Jumanji (1995),Adventure|Children|Fantasy,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0
2,3,Grumpier Old Men (1995),Comedy|Romance,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0


In [ ]:
df_ava_filme_cat_onehot = pd.merge( df_ava, df_filmes_cat_onehot, how='left', left_on='id_filme', right_on='id_filme' )
df_ava_filme_cat_onehot.head(10)

,id_usuario,id_filme,avaliacao,timestamp,data,nome,categoria,musical,thriller,adventure,war,animation,children,comedy,sci-fi,documentary,mystery,horror,film-noir,western,fantasy,drama,romance,action,crime
0,1,122,5.0,838985046,1996-08-02,Boomerang (1992),Comedy|Romance,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0
1,1,185,5.0,838983525,1996-08-02,"Net, The (1995)",Action|Crime|Thriller,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1
2,1,231,5.0,838983392,1996-08-02,Dumb & Dumber (1994),Comedy,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3,1,292,5.0,838983421,1996-08-02,Outbreak (1995),Action|Drama|Sci-Fi|Thriller,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0
4,1,316,5.0,838983392,1996-08-02,Stargate (1994),Action|Adventure|Sci-Fi,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0
5,1,329,5.0,838983392,1996-08-02,Star Trek: Generations (1994),Action|Adventure|Drama|Sci-Fi,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0
6,1,355,5.0,838984474,1996-08-02,"Flintstones, The (1994)",Children|Comedy|Fantasy,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0
7,1,356,5.0,838983653,1996-08-02,Forrest Gump (1994),Comedy|Drama|Romance|War,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,1,0,0
8,1,362,5.0,838984885,1996-08-02,"Jungle Book, The (1994)",Adventure|Children|Romance,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0
9,1,364,5.0,838983707,1996-08-02,"Lion King, The (1994)",Adventure|Animation|Children|Drama|Musical,1,0,1,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0


In [ ]:
df_generos_populares = df_ava_filme_cat_onehot.iloc[:, 8:].sum()
df_generos_populares

thriller       2584435
adventure      2121074
war             568063
animation       519112
children        820149
comedy         3934068
sci-fi         1490489
documentary     103454
mystery         630944
horror          768225
film-noir       131592
western         210459
fantasy        1028482
drama          4344198
romance        1901883
action         2845349
crime          1474957
dtype: int64

In [ ]:
print( df_generos_populares.sort_values(ascending=False).head() )
print( df_generos_populares.sort_values(ascending=False).tail() )

drama        4344198
comedy       3934068
action       2845349
thriller     2584435
adventure    2121074
dtype: int64
war            568063
animation      519112
western        210459
film-noir      131592
documentary    103454
dtype: int64


In [ ]:
df_generos_populares.sort_values(ascending=False).head().plot.barh()